# Ensemble level 2

In [1]:
import pandas as pd
import lightgbm as lgb
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
import numpy as np
import copy
from sklearn.svm import LinearSVC
from tffm import TFFMClassifier
import tensorflow as tf
from sklearn.ensemble import IsolationForest
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
import keras
from keras.layers import Dense, Dropout, Activation,Input
from keras.models import Sequential,Model
from keras.callbacks import EarlyStopping
from sklearn import preprocessing
import pickle
# data analysis

/home/douglas/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
#load or create your dataset
print('Load data...')
df_train = pd.read_csv("train.csv")
print(df_train.head())
df_test = pd.read_csv('test.csv')
ID = df_test['ID_code']
del df_test['ID_code']

# delete duplicated based on time
del df_train['ID_code']
df_train.drop_duplicates(keep='first', inplace=True)

# y_train output - deleted target value
y_train = df_train.target
del df_train['target']

# x_train
X_train = df_train.values

Load data...
   ID_code  target    var_0   var_1    var_2   var_3    var_4   var_5   var_6  \
0  train_0       0   8.9255 -6.7863  11.9081  5.0930  11.4607 -9.2834  5.1187   
1  train_1       0  11.5006 -4.1473  13.8588  5.3890  12.3622  7.0433  5.6208   
2  train_2       0   8.6093 -2.7457  12.0805  7.8928  10.5825 -9.0837  6.9427   
3  train_3       0  11.0604 -2.1518   8.9522  7.1957  12.5846 -1.8361  5.8428   
4  train_4       0   9.8369 -1.4834  12.8746  6.6375  12.2772  2.4486  5.9405   

     var_7   ...     var_190  var_191  var_192  var_193  var_194  var_195  \
0  18.6266   ...      4.4354   3.9642   3.1364   1.6910  18.5227  -2.3978   
1  16.5338   ...      7.6421   7.7214   2.5837  10.9516  15.4305   2.0339   
2  14.6155   ...      2.9057   9.7905   1.6704   1.6858  21.6042   3.1417   
3  14.9250   ...      4.4666   4.7433   0.7178   1.4214  23.0347  -1.2706   
4  19.2514   ...     -1.4905   9.5214  -0.1508   9.1942  13.2876  -1.5121   

   var_196  var_197  var_198  var_199

## LGBM model

In [ ]:
if (False):
    scaler = preprocessing.MinMaxScaler()
    scaler.fit(X_train)
    X_train = scaler.transform(X_train)

In [1]:
lgbmModels = []
lgbmACC = []
lgbmAUC = []

params = {
        'task': 'train',
        #'boosting_type': 'gbdt',
        'objective': 'xentropy',#'binary',xentropy;xentlambda
        'num_leaves': int(30),      # 31
        'learning_rate': np.exp(-4.108467241113353),   #0.01,
        'feature_fraction': 0.02720614438477189,#0.9,
        'bagging_fraction': 0.8350603586533163,#0.8,
        'bagging_freq': 1,
        'max_depth': 6,        #-1,
        'min_data_in_leaf': 13, #20,
        'lambda_l2': 4,        # 0,
        'is_unbalance' : False,
        'max_bin' : int(75),
        'verbose': -1
    }

NameError: name 'np' is not defined

In [7]:
kf = StratifiedKFold(n_splits=5, random_state=42) # all model have to use this 
np.random.seed(42)

for train_index, test_index in kf.split(X_train,y_train):
    print('test',train_index[0])
    for n in np.arange(20):      
        limit = 10000 # number of 1s 
        
        train_index = np.random.permutation(train_index)
        part1 = train_index[y_train[train_index] ==1] # values with 1
        part1 = np.random.permutation(part1)[0:limit]
        # same number of values with 0
              
        
        part2 = train_index[y_train[train_index] == 0]
        
        # building training data        
        train_indexMod = np.r_[ part1, part2[0:limit] ]
        
        
        lgb_train = lgb.Dataset(X_train[train_indexMod], y_train[train_indexMod]) 
        gbm = lgb.train(params,lgb_train, num_boost_round=4575, verbose_eval= 1)
        
        
        #lgb_train = lgb.Dataset(X_train[train_index], y_train[train_index]) 
        #gbm = lgb.train(params,lgb_train, num_boost_round=3688, verbose_eval= 1)
        y_pred = gbm.predict(X_train[test_index])
        #print('Ones', np.sum(y_pred > 0.5))
        
        rocData = roc_auc_score(y_train[test_index],y_pred)
        accData = accuracy_score(y_train[test_index],np.round(y_pred))
        
        print('n - iteration',n)
        print('ROC : ', rocData )
        print('ACC:', accData)
        #print(classification_report(y_train[test_index], np.round(y_pred)))
        lgbmACC.append(accData)
        lgbmAUC.append(rocData)
        lgbmModels.append(copy.deepcopy(gbm) )
        #print('ROC rounded: ', roc_auc_score(y_train[test_index],np.round(y_pred)))
        #print('ROC proba: ', roc_auc_score(y_train[test_index],gbm.predict_proba(X_train[test_index])))
        #indices = (y_train[test_index] == 1)
        #buff = y_train[test_index] 
        #print('ACCminorClass:', np.sum( np.round(y_pred[indices])==buff[indices] )/indices.shape )
        


test 39913
n - iteration 0
ROC :  0.8901408095289649
ACC: 0.8140046498837529
n - iteration 1
ROC :  0.8916050566212322
ACC: 0.8152546186345342
n - iteration 2
ROC :  0.890091495221151
ACC: 0.8132546686332842


KeyboardInterrupt: 

In [ ]:
#ploting values
print('ACC',np.mean(lgbmACC))
print('AUC', np.mean(lgbmAUC))

In [ ]:
#saving variables
file = open('lgbmModel', 'wb')
# dump information to that file
pickle.dump(lgbmModels, file)
# close the file
file.close()

## GBM

In [ ]:
gbmModels = []
gbmACC = []
gbmAUC = []

params = {
            'task': 'train',
            'booster': 'gbtree',
            'objective': 'binary:logistic',#'binary',
            'num_leaves': int(30),      # 31
            'learning_rate': np.exp(-4.108467241113353),   #0.01, Uses log values to improve optimization
            'colsample_bytree': 0.02720614438477189,#0.9,
            'subsample':  0.8350603586533163,#0.8,
            'max_depth': 6,        #-1,
            'min_child_weight': 10, #default=1,
            'lambda':4,        # 0,
            #'is_unbalance' : False,
            'max_bin' : int(75),
            #'nthread':3,
            #'verbosity': 0    
            'silent':1
        }

In [ ]:
kf = StratifiedKFold(n_splits=5, random_state=42) # all model have to use this 
np.random.seed(42)

for train_index, test_index in kf.split(X_train,y_train):
    print('test',train_index[0])
    
    if (train_index[0] == 0):
        #saving variables
        file = open('gbmModel', 'wb')
        # dump information to that file
        pickle.dump(gbmModels, file)
        # close the file
        file.close()
    
    for n in np.arange(20): 
        limit = 10000 # number of 1s 
        
        train_index = np.random.permutation(train_index)
        part1 = train_index[y_train[train_index] ==1] # values with 1
        part1 = np.random.permutation(part1)[0:limit]
        # same number of values with 0
        part2 = train_index[y_train[train_index] == 0]
        
        # building training data        
        train_indexMod = np.r_[ part1, part2[0:limit] ]        
        
        xgb_train = xgb.DMatrix(data = X_train[train_indexMod], label= y_train[train_indexMod]) 
        gbm = xgb.train(params,xgb_train, 4575)

        y_pred = gbm.predict(xgb.DMatrix(data =X_train[test_index]))
        
        rocData = roc_auc_score(y_train[test_index],y_pred)
        accData = accuracy_score(y_train[test_index],np.round(y_pred))
        
        print('n - iteration',n)
        print('ROC : ', rocData )
        print('ACC:', accData)
        #print(classification_report(y_train[test_index], np.round(y_pred)))
        gbmACC.append(accData)
        gbmAUC.append(rocData)
        gbmModels.append(copy.deepcopy(gbm) )
        #print('ROC rounded: ', roc_auc_score(y_train[test_index],np.round(y_pred)))
        #print('ROC proba: ', roc_auc_score(y_train[test_index],gbm.predict_proba(X_train[test_index])))
        #indices = (y_train[test_index] == 1)
        #buff = y_train[test_index] 
        #print('ACCminorClass:', np.sum( np.round(y_pred[indices])==buff[indices] )/indices.shape )
        
        

In [ ]:
#ploting values
print('ACC',np.mean(gbmACC))
print('AUC', np.mean(gbmAUC))

In [ ]:
#saving variables
file = open('gbmModel', 'wb')
# dump information to that file
pickle.dump(gbmModels, file)
# close the file
file.close()

# SVM

In [ ]:
if (True):
    scaler = preprocessing.StandardScaler().fit(X_train)
    X_train = scaler.transform(X_train)
    
    scaler = preprocessing.MinMaxScaler()
    scaler.fit(X_train)
    X_train = scaler.transform(X_train)

In [ ]:
svmModels = []
svmACC = []
svmAUC = []

In [ ]:
kf = StratifiedKFold(n_splits=5, random_state=42) # all model have to use this 
np.random.seed(42)

for train_index, test_index in kf.split(X_train,y_train):
    print('test',train_index[0])
    for n in np.arange(20): 
        limit = 10000 # number of 1s 
        train_index = np.random.permutation(train_index)
        part1 = train_index[y_train[train_index] ==1] # values with 1
        part1 = np.random.permutation(part1)[0:limit]
        # same number of values with 0
              
        
        part2 = train_index[y_train[train_index] == 0]
        
        # building training data        
        train_indexMod = np.r_[ part1, part2[0:limit] ]
    
        model = LinearSVC( C =1 )
        #clf = CalibratedClassifierCV(model) 
        model.fit(X_train[train_indexMod], y_train[train_indexMod])
    
        y_pred = model.predict(X_train[test_index])
                
        rocData = roc_auc_score(y_train[test_index],y_pred)
        accData = accuracy_score(y_train[test_index],np.round(y_pred))
        
        print('n - iteration',n)
        print('ROC : ', rocData )
        print('ACC:', accData)
        #print(classification_report(y_train[test_index], np.round(y_pred)))
        svmACC.append(accData)
        svmAUC.append(rocData)
        svmModels.append(copy.deepcopy(model) )

In [ ]:
#ploting values
print('ACC',np.mean(svmACC))
print('AUC', np.mean(svmAUC))

In [ ]:
#saving variables
file = open('svmModel', 'wb')
# dump information to that file
pickle.dump(svmModels, file)
# close the file
file.close()

# FM

In [ ]:
fmModels = []
fmACC = []
fmAUC = []        

In [ ]:
kf = StratifiedKFold(n_splits=5, random_state=42) # all model have to use this 
np.random.seed(42)

for train_index, test_index in kf.split(X_train,y_train):
    print('test',train_index[0])
    for n in np.arange(20): 
        limit = 10000 # number of 1s 
        train_index = np.random.permutation(train_index)
        part1 = train_index[y_train[train_index] ==1] # values with 1
        part1 = np.random.permutation(part1)[0:limit]
        # same number of values with 0
        
        ### improve the way I save the models
        model = TFFMClassifier(
                order=3,
                rank=10,
                optimizer=tf.train.AdamOptimizer(learning_rate=0.0001), 
                n_epochs=50, 
                batch_size=1024,
                init_std=0.01,
                reg=10.0,
                input_type='dense'
                )
        
        part2 = train_index[y_train[train_index] == 0]
        
        # building training data        
        train_indexMod = np.r_[ part1, part2[0:limit] ]
        
        model.fit(X_train, y_train, show_progress=False)
        y_pred = model.predict_proba(X_train[test_index])[:,1]
        
        rocData = roc_auc_score(y_train[test_index],y_pred)
        accData = accuracy_score(y_train[test_index],np.round(y_pred))
        
        print('n - iteration',n)
        print('ROC : ', rocData )
        print('ACC:', accData)
        #print(classification_report(y_train[test_index], np.round(y_pred)))
        fmACC.append(accData)
        fmAUC.append(rocData)
        fmModels.append(model)
        

In [ ]:
#ploting values
print('ACC',np.mean(fmACC))
print('AUC', np.mean(fmAUC))

In [ ]:
# salva FM models
val = 0
for modelo in fmModels:
    modelo.save_state('./fm/model'+ str(val) + '.tf')
    val+=1
#model.destroy()

#  Isolation forest

In [ ]:
clf = IsolationForest( max_samples=10000,
                      random_state=42, contamination=0.1,n_estimators = 1000,max_features=0.1)
isoModels = []

In [ ]:
kf = StratifiedKFold(n_splits=5, random_state=42) # all model have to use this 
np.random.seed(42)

for train_index, test_index in kf.split(X_train,y_train):
    print(train_index[0])
    clf.fit( X_train[train_index] )
    isoModels.append(clf)
#y_pred_train = clf.predict(X_train)

# Logistic Regression

In [ ]:
clf = LogisticRegression(random_state=0, solver='lbfgs',C = 0.1)
logModels = []
logACC = []
logAUC = []

In [ ]:
kf = StratifiedKFold(n_splits=5, random_state=42) # all model have to use this 
np.random.seed(42)

for train_index, test_index in kf.split(X_train,y_train):
    print('test',train_index[0])
    for n in np.arange(20): 
        limit = 10000 # number of 1s 
        train_index = np.random.permutation(train_index)
        part1 = train_index[y_train[train_index] ==1] # values with 1
        part1 = np.random.permutation(part1)[0:limit]
        # same number of values with 0
              
        
        part2 = train_index[y_train[train_index] == 0]
        
        # building training data        
        train_indexMod = np.r_[ part1, part2[0:limit] ]
    
        clf.fit(X_train[train_indexMod], y_train[train_indexMod])
        
        y_pred = clf.predict_proba(X_train[test_index])[:,1]
        
        rocData = roc_auc_score(y_train[test_index],y_pred)
        accData = accuracy_score(y_train[test_index],np.round(y_pred))
        
        print('n - iteration',n)
        print('ROC : ', rocData )
        print('ACC:', accData)
        #print(classification_report(y_train[test_index], np.round(y_pred)))
        logACC.append(accData)
        logAUC.append(rocData)
        logModels.append(clf)

In [ ]:
#ploting values
print('ACC',np.mean(logACC))
print('AUC', np.mean(logAUC))

In [ ]:
#saving variables
file = open('logModel', 'wb')
# dump information to that file
pickle.dump(logModels, file)
# close the file
file.close()

# Extra trees

In [ ]:
model = ExtraTreesClassifier(n_estimators=1000, min_samples_split=2, random_state=0,n_jobs=4)
extModels = []
extACC = []
extAUC = []

In [ ]:
kf = StratifiedKFold(n_splits=5, random_state=42) # all model have to use this 
np.random.seed(42)

for train_index, test_index in kf.split(X_train,y_train):
    print('test',train_index[0])
    for n in np.arange(20): 
        limit = 1000 # number of 1s 
        train_index = np.random.permutation(train_index)
        part1 = train_index[y_train[train_index] ==1] # values with 1
        part1 = np.random.permutation(part1)[0:limit]
        # same number of values with 0
        part2 = train_index[y_train[train_index] == 0]
        
        # building training data        
        train_indexMod = np.r_[ part1, part2[0:limit] ]
        
        model.fit(X_train[train_indexMod], y_train[train_indexMod])
        
        y_pred = model.predict_proba(X_train[test_index])[:,1]
        
        rocData = roc_auc_score(y_train[test_index],y_pred)
        accData = accuracy_score(y_train[test_index],np.round(y_pred))
        
        print('n - iteration',n)
        print('ROC : ', rocData )
        print('ACC:', accData)
        #print(classification_report(y_train[test_index], np.round(y_pred)))
        #print(classification_report(y_train[test_index], np.round(y_pred)))
        extACC.append(accData)
        extAUC.append(rocData)
        extModels.append(model)

# Random Forest

In [ ]:
model = RandomForestClassifier(n_estimators=1000, random_state=0,bootstrap=0.7,n_jobs=4)
rftModels = []
rftACC = []
rftAUC = []

In [ ]:
kf = StratifiedKFold(n_splits=5, random_state=42) # all model have to use this 
np.random.seed(42)

for train_index, test_index in kf.split(X_train,y_train):
    print('test',train_index[0])
    for n in np.arange(20): 
        limit = 10000 # number of 1s 
        train_index = np.random.permutation(train_index)
        part1 = train_index[y_train[train_index] ==1] # values with 1
        part1 = np.random.permutation(part1)[0:limit]
        # same number of values with 0
        part2 = train_index[y_train[train_index] == 0]
        
        # building training data        
        train_indexMod = np.r_[ part1, part2[0:limit] ]
        
        model.fit(X_train[train_indexMod], y_train[train_indexMod])
        
        y_pred = model.predict_proba(X_train[test_index])[:,1]
        
        rocData = roc_auc_score(y_train[test_index],y_pred)
        accData = accuracy_score(y_train[test_index],np.round(y_pred))
        
        print('n - iteration',n)
        print('ROC : ', rocData )
        print('ACC:', accData)
        #print(classification_report(y_train[test_index], np.round(y_pred)))

        rftACC.append(accData)
        rftAUC.append(rocData)
        rftModels.append(model)

# TSNE

In [ ]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

def tsne_plot(x1, y1, name="graph.png"):
    tsne = TSNE(n_components=2, random_state=0)
    X_t = tsne.fit_transform(x1)

    plt.figure(figsize=(12, 8))
    plt.scatter(X_t[np.where(y1 == 0), 0], X_t[np.where(y1 == 0), 1], marker='o', color='g', linewidth='1', alpha=0.8, label='Non Fraud')
    plt.scatter(X_t[np.where(y1 == 1), 0], X_t[np.where(y1 == 1), 1], marker='o', color='r', linewidth='1', alpha=0.8, label='Fraud')

    plt.legend(loc='best');
    #plt.savefig(name);
    plt.show();
    
tsne_plot(X_train[0:4000], y_train[0:4000], "original.png")

#  Autoencoders

In [ ]:
model = Sequential()
model.add(Dense(40, input_dim= X_train.shape[1],activation='tanh'))
model.add(Dropout(0.5))
model.add(Dense(20, activation='tanh'))
model.add(Dropout(0.25))
model.add(Dense(2, activation='linear'))
model.add(Dense(5, activation='tanh'))
model.add(Dense(20, activation='tanh'))
model.add(Dense(X_train.shape[1], activation='linear'))
        
        
model.compile(#loss=custom_objective,
               loss = 'binary_crossentropy',
               optimizer=keras.optimizers.SGD(lr=0.0001),
               metrics=['MSE'])

In [ ]:

scaler = preprocessing.StandardScaler().fit(X_train)
X_scale = scaler.transform(X_train)

scaler = preprocessing.MinMaxScaler()
scaler.fit(X_scale) # X_train
X_scale = scaler.transform(X_scale) # X_train
'''
# Standarize
scaler = preprocessing.StandardScaler().fit(X_train)
X_scale = scaler.transform(X_train)
'''
#
model.fit(X_scale[y_train == 1], X_scale[y_train == 1],
                  batch_size=128,
                  epochs=500,
                  verbose=1,validation_split = 0.20)

In [ ]:
hidden_representation = Sequential()
hidden_representation.add(model.layers[0])
hidden_representation.add(model.layers[1])
hidden_representation.add(model.layers[2])
hidden_representation.add(model.layers[3])
hidden_representation.add(model.layers[4])
hidden_representation.summary()

In [ ]:
x_norm, x_fraud = X_scale[y == 0], X_scale[y == 1]

norm_hid_rep = hidden_representation.predict(x_norm)
fraud_hid_rep = hidden_representation.predict(x_fraud)
y_n = np.zeros(norm_hid_rep.shape[0])
y_f = np.ones(fraud_hid_rep.shape[0])
rep_y = np.append(y_n, y_f)

plt.figure(figsize=(12, 8))
plt.scatter(norm_hid_rep[:,0], norm_hid_rep[:,1], marker='o', color='g', linewidth='1', alpha=0.8, label='Non Fraud')
plt.scatter(fraud_hid_rep[:,0],fraud_hid_rep[:,1], marker='o', color='r', linewidth='1', alpha=0.8, label='Fraud')

plt.legend(loc='best');
#plt.savefig(name);
plt.show();


# level  2

# lgbm load/predict

In [ ]:
# open a file, where you stored the pickled data
file = open('lgbmModel', 'rb')
# dump information to that file
l_lgbmModels = pickle.load(file)
# close the file
file.close()

In [ ]:
kf = StratifiedKFold(n_splits=5, random_state=42) # all model have to use this 
np.random.seed(42)

ind = 0
vec = []

for train_index, test_index in kf.split(X_train,y_train):
    
    acum = 0
    for n in np.arange(20):
        y_pred = l_lgbmModels[ind*20+n].predict(X_train[test_index])
        acum += y_pred
    
    acum= acum/20 # get the mean value
    
    vec = np.r_[vec,acum]
    ind = ind+1

In [ ]:
#saving variables
file = open('lgbmPrevTest', 'wb')
# dump information to that file
pickle.dump(vec, file)
# close the file
file.close()

## gbm load/predict

In [ ]:
#saving variables
file = open('gbmModel', 'rb')
# dump information to that file
l_gbmModels = pickle.load(file)
# close the file
file.close()

In [ ]:
kf = StratifiedKFold(n_splits=5, random_state=42) # all model have to use this 
np.random.seed(42)

ind = 0
vec = []

for train_index, test_index in kf.split(X_train,y_train):
    
    acum = 0
    for n in np.arange(20):
        y_pred = l_gbmModels[ind*20+n].predict(xgb.DMatrix(data =X_train[test_index]))
        acum += y_pred
    
    acum= acum/20 # get the mean value
    
    vec = np.r_[vec,acum]
    ind = ind+1

In [ ]:
#saving variables
file = open('gbmPrevTest', 'wb')
# dump information to that file
pickle.dump(vec, file)
# close the file
file.close()

## svm load/predict

In [ ]:
#saving variables
file = open('svmModel', 'rb')
# dump information to that file
l_svmModels = pickle.load(file)
# close the file
file.close()

In [ ]:
scaler = preprocessing.StandardScaler().fit(X_train)
X_trains = scaler.transform(X_train)
    
scaler = preprocessing.MinMaxScaler()
scaler.fit(X_trains)
X_train = scaler.transform(X_trains)

In [ ]:
kf = StratifiedKFold(n_splits=5, random_state=42) # all model have to use this 
np.random.seed(42)

ind = 0
vec = []

for train_index, test_index in kf.split(X_train,y_train):
    
    acum = 0
    for n in np.arange(20):
        y_pred = l_svmModels[ind*20+n].decision_function(X_trains[test_index])
        acum += y_pred
    
    acum= acum/20 # get the mean value
    
    vec = np.r_[vec,acum]
    ind = ind+1

In [ ]:
#saving variables
file = open('svmPrevTest', 'wb')
# dump information to that file
pickle.dump(vec, file)
# close the file
file.close()

## FM load/save

In [ ]:
#load fm model
l_fmModels = []
for n in np.arange(0,100):
    model = TFFMClassifier(
                    order=3,
                    rank=10,
                    optimizer=tf.train.AdamOptimizer(learning_rate=0.0001), 
                    n_epochs=50, 
                    batch_size=1024,
                    init_std=0.01,
                    reg=10.0,
                    input_type='dense'
                    )
    model.core.set_num_features(X_train.shape[1])
    model.load_state('./fm/model'+str(n)+'.tf')
    l_fmModels.append(model)

In [ ]:
kf = StratifiedKFold(n_splits=5, random_state=42) # all model have to use this 
np.random.seed(42)

ind = 0
vec = []

for train_index, test_index in kf.split(X_train,y_train):
    
    acum = 0
    for n in np.arange(20):
        y_pred = l_fmModels[ind*20+n].predict_proba(X_train[test_index])[:,1]
        acum += y_pred
    
    acum= acum/20 # get the mean value
    
    vec = np.r_[vec,acum]
    ind = ind+1

In [ ]:
#saving variables
file = open('fmPrevTest', 'wb')
# dump information to that file
pickle.dump(vec, file)
# close the file
file.close()

## log load/save

In [ ]:
#saving variables
file = open('logModel', 'rb')
# dump information to that file
l_logModels = pickle.load(file)
# close the file
file.close()

In [ ]:
kf = StratifiedKFold(n_splits=5, random_state=42) # all model have to use this 
np.random.seed(42)

ind = 0
vec = []

for train_index, test_index in kf.split(X_train,y_train):
    
    acum = 0
    for n in np.arange(20):
        y_pred = l_logModels[ind*20+n].predict_proba(X_train[test_index])[:,1]
        acum += y_pred
    
    acum= acum/20 # get the mean value
    
    vec = np.r_[vec,acum]
    ind = ind+1

In [ ]:
#saving variables
file = open('logPrevTest', 'wb')
# dump information to that file
pickle.dump(vec, file)
# close the file
file.close()

# Neural model

In [ ]:
file = open('lgbmPrevTest', 'rb')
yprev_lgbmPrev = pickle.load(file)
file.close()

file = open('gbmPrevTest', 'rb')
yprev_gbmPrev = pickle.load(file)
file.close()

file = open('svmPrevTest', 'rb')
yprev_svmPrev = pickle.load(file)
file.close()

file = open('fmPrevTest', 'rb')
yprev_fmPrev = pickle.load(file)
file.close()

file = open('logPrevTest', 'rb')
yprev_logPrev = pickle.load(file)
file.close()

In [ ]:
kf = StratifiedKFold(n_splits=5, random_state=42) # all model have to use this 
np.random.seed(42)
# Concatenates values
y_prev = np.c_[yprev_lgbmPrev,yprev_gbmPrev,yprev_svmPrev,yprev_fmPrev,yprev_logPrev]

y_teste = []
for train_index, test_index in kf.split(X_train,y_train):
    y_teste = np.r_[y_teste,y_train[test_index]]

In [ ]:
kf = StratifiedKFold(n_splits=5, random_state=42) # all model have to use this 
np.random.seed(42)

ind = 0
vec = []

for train_index, test_index in kf.split(y_prev,y_teste):
    for n in np.arange(1):
        model = Sequential()
        model.add(Dense(20, input_dim= y_prev.shape[1],activation='tanh'))
        model.add(Dropout(0.5))
        model.add(Dense(8, activation='tanh'))
        model.add(Dropout(0.25))
        model.add(Dense(2, activation='softmax'))
        
        
        model.compile(#loss=custom_objective,
                      loss = 'binary_crossentropy',#'binary_crossentropy',
                      optimizer=keras.optimizers.SGD(lr=0.0001)
                      )
                      #metrics=['accuracy'])
        
        
        
        epsilon = 0.1
        y_train2 = copy.deepcopy(y_teste)
        y_train2[y_train == 1] = 1-epsilon
        y_train2[y_train == 0] = epsilon
        
        y_final = np.c_[y_train2, 1-y_train2]
        
        earlyStopping=keras.callbacks.EarlyStopping(monitor='roc_auc_score', patience=3, verbose=0, mode='auto')
        
        #model.fit(y_prev[train_indexMod], [y_train[train_indexMod],1-y_train[train_indexMod] ],
        model.fit(y_prev[train_index], y_final[train_index],
                  batch_size=128,
                  epochs=50,
                  callbacks=[earlyStopping], 
                  validation_split=0.2,
                  verbose=1)
        
        y_fold = model.predict(y_prev[test_index])[:,0]
        
        rocData = roc_auc_score(y_train[test_index],y_fold)
        accData = accuracy_score(y_train[test_index],np.round(y_fold))
        
        print('n - iteration',n)
        print('ROC : ', rocData )
        print('ACC:', accData)
        print(classification_report(y_train[test_index], np.round(y_fold)))

# Semi test

In [133]:
X_train.shape

(200000, 200)

In [145]:
lgbmModels100 = []
lgbmACC100 = []
lgbmAUC100 = []

lgbmModels200 = []
lgbmACC200 = []
lgbmAUC200 = []

params = {
        'task': 'train',
        #'boosting_type': 'gbdt',
        'objective': 'xentropy',#'binary',xentropy;xentlambda
        'num_leaves': int(30),      # 31
        'learning_rate': np.exp(-4.108467241113353),   #0.01,
        'feature_fraction': 0.02720614438477189,#0.9,
        'bagging_fraction': 0.8350603586533163,#0.8,
        'bagging_freq': 1,
        'max_depth': 6,        #-1,
        'min_data_in_leaf': 13, #20,
        'lambda_l2': 4,        # 0,
        'is_unbalance' : True,
        'max_bin' : int(75),
        'verbose': -1
    }

In [146]:
kf = StratifiedKFold(n_splits=5, random_state=42) # all model have to use this 
np.random.seed(42)

for train_index, test_index in kf.split(X_train,y_train):
        
        print('Part 1')
        #lgb_train = lgb.Dataset(X_train[train_index,0:100], y_train[train_index]) 
        lgb_train= lgb.Dataset(np.r_[X_train[train_index],df_test.values[kp,:]], np.r_[y_train[train_index],np.ones(len(kp))]) 
        gbm = lgb.train(params,lgb_train, num_boost_round=4575, verbose_eval= 1)
        
        y_pred = gbm.predict(X_train[test_index,0:100])
        #print('Ones', np.sum(y_pred > 0.5))
        
        rocData = roc_auc_score(y_train[test_index],y_pred)
        accData = accuracy_score(y_train[test_index],np.round(y_pred))
        
        #print('n - iteration',n)
        print('ROC : ', rocData )
        print('ACC:', accData)
        #print(classification_report(y_train[test_index], np.round(y_pred)))
        lgbmACC100.append(accData)
        lgbmAUC100.append(rocData)
        lgbmModels100.append(copy.deepcopy(gbm) )

        print('Part 2')        
        
        #lgb_train = lgb.Dataset(X_train[train_index,100:], y_train[train_index]) 
        #lgb_train = lgb.Dataset(X_train[train_index], y_train[train_index]) 
        lgb_train= lgb.Dataset(np.r_[X_train[train_index],df_test.values[jp,:]], np.r_[y_train[train_index],np.ones(len(jp))]) 
        gbm = lgb.train(params,lgb_train, num_boost_round=4575, verbose_eval= 1)
        y_pred = gbm.predict(X_train[test_index,100:])
        #print('Ones', np.sum(y_pred > 0.5))
        
        rocData = roc_auc_score(y_train[test_index],y_pred)
        accData = accuracy_score(y_train[test_index],np.round(y_pred))
        
        #print('n - iteration',n)
        print('ROC : ', rocData )
        print('ACC:', accData)
        #print(classification_report(y_train[test_index], np.round(y_pred)))
        lgbmACC200.append(accData)
        lgbmAUC200.append(rocData)
        lgbmModels200.append(copy.deepcopy(gbm) )
        

Part 1
ROC :  0.8121584830357537
ACC: 0.10049748756281093
Part 2
ROC :  0.5173494759049864
ACC: 0.10049748756281093
Part 1
ROC :  0.8116324314892008
ACC: 0.10049748756281093
Part 2
ROC :  0.5209958932167211
ACC: 0.10049748756281093
Part 1
ROC :  0.8132956603862289
ACC: 0.1005
Part 2
ROC :  0.5033356840035509
ACC: 0.1005
Part 1
ROC :  0.811990142432119
ACC: 0.10047751193779844
Part 2
ROC :  0.5146125802383094
ACC: 0.10047751193779844
Part 1
ROC :  0.8142798914715966
ACC: 0.10047751193779844
Part 2
ROC :  0.5031979904790765
ACC: 0.10047751193779844


In [137]:
np.mean(lgbmAUC200)

0.8994682949027432

In [30]:
acum =[]
for m in lgbmModels100:
    
    y_pred = m.predict(df_test.values[:,0:100])
    
    if len(acum):
        acum = np.c_[acum,y_pred]
    else:
        acum = copy.deepcopy(y_pred)

In [52]:
(np.sum(acum,axis=1)/5) > 0.5

1751

In [53]:
acum2 =[]
for m in lgbmModels200:
    
    y_pred = m.predict(df_test.values[:,100:])
    
    if len(acum2):
        acum2 = np.c_[acum,y_pred]
    else:
        acum2 = copy.deepcopy(y_pred)

In [111]:
np.sum((np.sum(acum2,axis=1)/5) > 0.5)

2211

# Second round training

In [83]:
kp = ((np.sum(acum,axis=1)/5) > 0.5)*np.arange(0,200000)
kp = kp[kp>0]
kn = ((np.sum(acum,axis=1)/5) < 0.05)*np.arange(0,200000)
kn = kn[kn>0]

In [84]:
jp = ((np.sum(acum2,axis=1)/5) > 0.5)*np.arange(0,200000)
jp = jp[jp>0]
jf = ((np.sum(acum2,axis=1)/5) < 0.05)*np.arange(0,200000)
jf = jf[jf>0]

In [99]:
tee1 = np.intersect1d(jp,kp)
tee2 = np.intersect1d(kn,jf)

In [114]:
kp.shape

(1751,)

In [93]:
lgbm2Models100 = []
lgbm2ACC100 = []
lgbm2AUC100 = []

lgbm2Models200 = []
lgbm2ACC200 = []
lgbm2AUC200 = []

params = {
        'task': 'train',
        #'boosting_type': 'gbdt',
        'objective': 'xentropy',#'binary',xentropy;xentlambda
        'num_leaves': int(30),      # 31
        'learning_rate': np.exp(-4.108467241113353),   #0.01,
        'feature_fraction': 0.02720614438477189,#0.9,
        'bagging_fraction': 0.8350603586533163,#0.8,
        'bagging_freq': 1,
        'max_depth': 6,        #-1,
        'min_data_in_leaf': 13, #20,
        'lambda_l2': 4,        # 0,
        'is_unbalance' : False,
        'max_bin' : int(75),
        'verbose': -1
    }

In [97]:
kf = StratifiedKFold(n_splits=5, random_state=42) # all model have to use this 
np.random.seed(42)

for train_index, test_index in kf.split(X_train,y_train):
        
        print('Part 1')
        lgb_train = lgb.Dataset(np.r_[X_train[train_index,0:100],df_test.values[jp,0:100],df_test.values[jf,0:100]], np.r_[y_train[train_index],np.ones(len(jp)),np.zeros(len(jf))]) 
        #lgb_train = lgb.Dataset(np.r_[X_train[train_index,0:100],df_test.values[jp,0:100]], np.r_[y_train[train_index],np.ones(len(jp))])  
        
        gbm = lgb.train(params,lgb_train, num_boost_round=4575, verbose_eval= 1)
        
        y_pred = gbm.predict(X_train[test_index,0:100])
        #print('Ones', np.sum(y_pred > 0.5))
        
        rocData = roc_auc_score(y_train[test_index],y_pred)
        accData = accuracy_score(y_train[test_index],np.round(y_pred))
        
        #print('n - iteration',n)
        print('ROC : ', rocData )
        print('ACC:', accData)
        #print(classification_report(y_train[test_index], np.round(y_pred)))
        lgbm2ACC100.append(accData)
        lgbm2AUC100.append(rocData)
        lgbm2Models100.append(copy.deepcopy(gbm) )

        print('Part 2')        
        
        lgb_train = lgb.Dataset(np.r_[X_train[train_index,100:],df_test.values[kp,100:],df_test.values[kn,100:]], np.r_[y_train[train_index],np.ones(len(kp)),np.zeros(len(kn))]) 
        #lgb_train = lgb.Dataset(np.r_[X_train[train_index,100:],df_test.values[kp,100:]], np.r_[y_train[train_index],np.ones(len(kp))]) 
        gbm = lgb.train(params,lgb_train, num_boost_round=4575, verbose_eval= 1)
        y_pred = gbm.predict(X_train[test_index,100:])
        #print('Ones', np.sum(y_pred > 0.5))
        
        rocData = roc_auc_score(y_train[test_index],y_pred)
        accData = accuracy_score(y_train[test_index],np.round(y_pred))
        
        #print('n - iteration',n)
        print('ROC : ', rocData )
        print('ACC:', accData)
        #print(classification_report(y_train[test_index], np.round(y_pred)))
        lgbm2ACC200.append(accData)
        lgbm2AUC200.append(rocData)
        lgbm2Models200.append(copy.deepcopy(gbm) )
        

Part 1
ROC :  0.8133114063378669
ACC: 0.907027324316892
Part 2
ROC :  0.8161975758073533
ACC: 0.9035774105647358
Part 1
ROC :  0.8137991222841353
ACC: 0.907102322441939
Part 2
ROC :  0.8059949343081984
ACC: 0.9030524236894077
Part 1
ROC :  0.8155865198742253
ACC: 0.907825
Part 2
ROC :  0.818125727670707
ACC: 0.903
Part 1
ROC :  0.8175853343090581
ACC: 0.9064226605665142
Part 2
ROC :  0.814995281584237
ACC: 0.9028725718142954
Part 1
ROC :  0.8177827774989312
ACC: 0.9077976949423736
Part 2
ROC :  0.8203408047461053
ACC: 0.9030725768144203


In [92]:
print(np.mean(lgbmAUC200),np.mean(lgbm2AUC200))
print(np.mean(lgbmAUC100),np.mean(lgbm2AUC100))


0.815235050567173 0.8151308648233202
0.811700174446727 0.8152294277736806


In [98]:
print(np.mean(lgbmAUC200),np.mean(lgbm2AUC200))
print(np.mean(lgbmAUC100),np.mean(lgbm2AUC100))

0.815235050567173 0.8152785819532303
0.811700174446727 0.8151057485335447


# Calibration Curves

In [ ]:
from sklearn.calibration import calibration_curve
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
ax1 = plt.subplot2grid((3, 1), (0, 0), rowspan=2)

ax1.plot([0, 1], [0, 1], "k:", label="Perfectly calibrated")

fraction_of_positives, mean_predicted_value = calibration_curve(y_train, yprev_lgbmPrev, n_bins=10)
ax1.plot(mean_predicted_value, fraction_of_positives, "s-",label="%s" % ('lgbm', ))

fraction_of_positives, mean_predicted_value = calibration_curve(y_train, yprev_fmPrev, n_bins=10)
ax1.plot(mean_predicted_value, fraction_of_positives, "s-",label="%s" % ('fm', ))

fraction_of_positives, mean_predicted_value = calibration_curve(y_train, yprev_gbmPrev, n_bins=10)
ax1.plot(mean_predicted_value, fraction_of_positives, "s-",label="%s" % ('gbm', ))

fraction_of_positives, mean_predicted_value = calibration_curve(y_train, yprev_logPrev, n_bins=10)
ax1.plot(mean_predicted_value, fraction_of_positives, "s-",label="%s" % ('log', ))
ax1.legend(loc="lower right")

ax1.set_ylabel("Fraction of positives")
ax1.set_ylim([-0.05, 1.05])
ax1.legend(loc="lower right")
ax1.set_title('Calibration plots  (reliability curve)')

In [ ]:
# ROC curve

In [ ]:
from sklearn.metrics import roc_curve,auc

fpr = dict()
tpr = dict()
roc_auc = dict()
for i in np.arange(0,5):
    fpr[i], tpr[i], _ = roc_curve(y_train, y_prev[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])
    
#fpr["micro"], tpr["micro"], _ = roc_curve(y_train.ravel(), y_prev.ravel())
#roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

In [ ]:
#yprev_lgbmPrev,yprev_gbmPrev,yprev_svmPrev,yprev_fmPrev,yprev_logPrev
plt.figure()
lw = 2

plt.plot(fpr[0], tpr[0], color='black',
         lw=lw, label='ROC curve GBM (area = %0.2f)' % roc_auc[0])

plt.plot(fpr[1], tpr[1], color='green',
         lw=lw, label='ROC curve LGBM (area = %0.2f)' % roc_auc[1])

plt.plot(fpr[2], tpr[2], color='darkorange',
         lw=lw, label='ROC curve svm (area = %0.2f)' % roc_auc[2])

plt.plot(fpr[3], tpr[3], color='red',
         lw=lw, label='ROC curve fm (area = %0.2f)' % roc_auc[3])

plt.plot(fpr[4], tpr[4], color='gray',
         lw=lw, label='ROC curve log (area = %0.2f)' % roc_auc[4])

plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()

# Test predicting

In [ ]:
# x_train
X_test = df_test.values

In [ ]:
file = open('lgbmModel', 'rb')
# dump information to that file
l_lgbmModels = pickle.load(file)
# close the file
file.close()

acum =0
for n in np.arange(100):
    y_pred = l_lgbmModels[n].predict(X_test)
    acum += y_pred
    
acum= acum/100 # get the mean value

#saving variables
file = open('lgbmPrevFinal', 'wb')
# dump information to that file
pickle.dump(acum, file)
# close the file
file.close()    

In [ ]:
#saving variables
file = open('gbmModel', 'rb')
# dump information to that file
l_gbmModels = pickle.load(file)
# close the file
file.close()

acum =0
for n in np.arange(100):
    y_pred = l_gbmModels[n].predict(xgb.DMatrix(data =X_test))
    acum += y_pred
    
acum= acum/100 # get the mean value

#saving variables
file = open('gbmPrevFinal', 'wb')
# dump information to that file
pickle.dump(acum, file)
# close the file
file.close()   

In [ ]:
#saving variables
file = open('svmModel', 'rb')
# dump information to that file
l_svmModels = pickle.load(file)
# close the file
file.close()

scaler = preprocessing.StandardScaler().fit(X_train)
X_trains = scaler.transform(X_train)
X_tests = scaler.transform(X_test)
    
scaler = preprocessing.MinMaxScaler()
scaler.fit(X_trains)
X_tests = scaler.transform(X_tests)

acum =0
for n in np.arange(100):
    y_pred = l_svmModels[n].decision_function(X_tests)
    acum += y_pred
    
acum= acum/100 # get the mean value

#saving variables
file = open('svmPrevFinal', 'wb')
# dump information to that file
pickle.dump(acum, file)
# close the file
file.close()  

In [ ]:
#load fm model
'''    
l_fmModels = []
for n in np.arange(0,100):
    model = TFFMClassifier(
                    order=3,
                    rank=10,
                    optimizer=tf.train.AdamOptimizer(learning_rate=0.0001), 
                    n_epochs=50, 
                    batch_size=1024,
                    init_std=0.01,
                    reg=10.0,
                    input_type='dense'
                    )
    model.core.set_num_features(X_train.shape[1])
    model.load_state('./fm/model'+str(n)+'.tf')
    l_fmModels.append(model)

'''
acum =[]
for n in np.arange(100):   
    y_pred = l_fmModels[n].predict_proba(X_test)[:,1]
    if len(acum):
        acum = np.c_[acum,y_pred]
    else:
        acum = copy.deepcopy(y_pred)
'''    
acum= acum/100 # get the mean value

#saving variables
file = open('fmPrevFinal', 'wb')
# dump information to that file
pickle.dump(acum, file)
# close the file
file.close()  
'''

In [ ]:
acum1 = np.mean(acum,axis=1)
acum2 = np.median(acum,axis=1)

In [ ]:
np.sum(yprev_fmPrev>0.5)

In [ ]:
fm1 = np.mean(acum[:,0:20],axis=1)
fm2 = np.mean(acum[:,20:40],axis=1)
fm3 = np.mean(acum[:,40:60],axis=1)
fm4 = np.mean(acum[:,60:80],axis=1)
fm5 = np.mean(acum[:,80:100],axis=1)
end = np.c_[fm1,fm2,fm3,fm4,fm5]

In [ ]:
np.sum(end>0.5,axis=0)

## Final ensemble

In [ ]:
file = open('lgbmPrevFinal', 'rb')
lgbmPrevFinal = pickle.load(file)
file.close()
file = open('gbmPrevFinal', 'rb')
gbmPrevFinal = pickle.load(file)
file.close()
file = open('svmPrevFinal', 'rb')
svmPrevFinal = pickle.load(file)
file.close()
file = open('fmPrevFinal', 'rb')
fmPrevFinal = pickle.load(file)
file.close()

In [ ]:
svmPrevFinal2 = svmPrevFinal-(min(svmPrevFinal))
svmPrevFinal2 = svmPrevFinal2/max(svmPrevFinal2)

y_prev = np.c_[lgbmPrevFinal,gbmPrevFinal,svmPrevFinal2,fmPrevFinal]

In [ ]:
mat1 = np.array([0.12659275, 0.21944339, 0.35109897, 0.30286489])
geometric = (y_prev*mat1)
geometric = geometric[:,0]*geometric[:,1]*geometric[:,2]*geometric[:,3]

In [ ]:
mat2 = np.array([0.44948398, 0.13951325, 0.07288259, 0.33812018])
meanV = (y_prev*mat2)
meanV = np.sum(meanV,axis=1)

In [ ]:
y_prev

In [ ]:
meanV 

In [ ]:
meanV = np.sqrt(lgbmPrevFinal*gbmPrevFinal)
meanV

In [ ]:
submission = pd.concat([ID, pd.DataFrame(meanV)],axis=1)#,columns=['session_id', 'is_click'])
submission.columns = ['ID_code', 'target']
submission.to_csv('beforeIgo.csv',index=False)

In [ ]:
meanV = (lgbmPrevFinal+gbmPrevFinal)/2
meanV

In [ ]:
submission = pd.concat([ID, pd.DataFrame(meanV)],axis=1)#,columns=['session_id', 'is_click'])
submission.columns = ['ID_code', 'target']
submission.to_csv('mean2.csv',index=False)

In [ ]:
np.sum(meanV > 0.5)

In [ ]:
sub2 = np.power(lgbmPrevFinal*gbmPrevFinal*fmPrevFinal,1/3)

In [ ]:
np.sum(sub2>0.5)

In [ ]:
submission = pd.concat([ID, pd.DataFrame(sub2)],axis=1)#,columns=['session_id', 'is_click'])
submission.columns = ['ID_code', 'target']
submission.to_csv('sumData2.csv',index=False)

#  Crazy codes

In [ ]:
max(y_prev[:,2])
k = y_prev[:,2]

k =  k - min(k)
k = k / max(k)
y_prev[:,2] = 1-k




In [ ]:
final = np.mean(y_prev,axis = 1)            

submission = pd.concat([ID, pd.DataFrame(final)],axis=1)#,columns=['session_id', 'is_click'])
submission.columns = ['ID_code', 'target']
submission.to_csv('meanVal.csv',index=False)

In [ ]:
hardCode = np.round(y_prev[])

In [ ]:
np.sum(hardCode[:,1:],axis=1)

In [ ]:
final = np.mean(y_prev[:,0:2],axis = 1) 
#yprev_lgbmPrev,yprev_gbmPrev,yprev_svmPrev,yprev_fmPrev,yprev_logPrev
rocData = roc_auc_score(y_teste,final)
rocData

In [ ]:
jjj = []
jkk = []
for x in np.arange(0,10000):
    i = np.array([random.random(),random.random(),random.random(),random.random(),random.random()])
    i = i/np.sum(i)
    # np.power(values, 1/3)
    y_prevj = y_prev*i
    rocData = roc_auc_score(y_teste,np.power(y_prevj[:,0]*y_prevj[:,1]*y_prevj[:,2]*y_prevj[:,3]*y_prevj[:,4],5))
    jj.append(rocData)
    jk.append(i)
    
    
    rocData = roc_auc_score(y_teste,np.mean(y_prevj,axis=1))
    jjj.append(rocData)
    jkk.append(i)
    #print(i)
    #print(rocData)
print(max(jj))
print(jk[np.argmax(jj)])
print('2')
print(max(jjj))
print(jkk[np.argmax(jjj)])

In [ ]:
print(max(jj))
print(jk[np.argmax(jj)])
print('2')
print(max(jjj))
print(jkk[np.argmax(jjj)])

In [ ]:
import random
    np.array[random.random(),random.random(),random.random(),random.random(),random.random()]

In [ ]:
y_prev*i

In [ ]:
y_prev

In [ ]:
jk[np.argmax(jj)]

In [ ]:
values = np.array([0.39823669, 0.14117335, 0.09870075, 0.34496129, 0.01692792]) # 0.9035755088777733

In [ ]:
np.power(values, 1/3)

In [ ]:
0.9014410654261418
[0.12659275 0.21944339 0.33370293 0.30286489 0.01739604]
2
0.9041437196865552
[0.44436681 0.13951325 0.07288259 0.33812018 0.00511717]


In [ ]:
file = open('svmPrevTest', 'rb')
yprev_svmPrev = pickle.load(file)
file.close()

In [ ]:
min(yprev_svmPrev)

In [ ]:
max(yprev_svmPrev)

In [ ]:
indexes = (y_train==1)
a = (np.round(yprev_lgbmPrev[indexes]) == 1)

b = (np.round(yprev_gbmPrev[indexes]) == 1)

k = (np.sign(yprev_svmPrev))
k[k<-0.5] = 0
k
c = (np.round(k[indexes]) == 1)

d = (np.round(yprev_fmPrev[indexes]) == 1)

e = (np.round(yprev_logPrev[indexes])  == 1)


In [ ]:
#a= 0
p = (a.astype(int)+b.astype(int)+c.astype(int)+d.astype(int)+e.astype(int))
p = (b.astype(int)+c.astype(int)+d.astype(int)+e.astype(int))

In [ ]:
np.sum(p == 4)

In [ ]:
np.sum(a != (p==1))

In [ ]:
np.sum(k)

In [ ]:
np.sum(np.round(yprev_lgbmPrev))

In [ ]:
p >3

In [ ]:
k = (np.sign(yprev_svmPrev))
k[k<-0.5] = 0
k